In [5]:
import numpy as np
import pandas as pd
import cv2
import os

import shutil
from pathlib import Path

import torch
import torch.nn as nn
import torchvision as tv
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

if torch.cuda.is_available():
    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
    device = torch.device("cuda")
elif torch.cuda.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# import imutils
from tqdm.autonotebook import tqdm

RANDOM_SEED = 123

In [6]:
yes_dir = "./dataset/yes"
no_dir = "./dataset/no"
IMG_SIZE = (224, 224)

In [7]:
# move
TRAIN_DIR = Path("./dataset/TRAIN")
VAL_DIR = Path("./dataset/VAL")
TEST_DIR = Path("./dataset/TEST")

TRAIN_DIR_CROPED = Path("./dataset/TRAIN_CROPED")
VAL_DIR_CROPED = Path("./dataset/VAL_CROPED")
TEST_DIR_CROPED = Path("./dataset/TEST_CROPED")


for path in [TRAIN_DIR, VAL_DIR, TEST_DIR, TRAIN_DIR_CROPED, VAL_DIR_CROPED, TEST_DIR_CROPED]:
    for label in ["yes", "no"]:
        d = path / label
        if not d.exists():
            d.mkdir(parents=True)

In [10]:
train_dataset = datasets.ImageFolder(
    root=TRAIN_DIR_CROPED,
    transform=transforms.Compose(
        [
            transforms.Resize(IMG_SIZE),
            # transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(p=0.2),
            transforms.RandomVerticalFlip(p=0.2),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
)

val_dataset = datasets.ImageFolder(
    root=VAL_DIR_CROPED,
    transform=transforms.Compose(
        [
            transforms.Resize(IMG_SIZE),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
)

train_dataloader = DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=1, pin_memory=True
)

val_dataloader = DataLoader(
    val_dataset, batch_size=16, shuffle=True, num_workers=1, pin_memory=True
)

In [11]:
for i, (images, targets) in enumerate(train_dataloader):
    print(images.shape, targets.shape)
    print(targets.numpy())
    print(images[0].numpy())
    break

torch.Size([32, 3, 224, 224]) torch.Size([32])
[1 0 1 0 1 0 0 0 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1]
[[[-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  ...
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]]

 [[-2.0357141 -2.0357141 -2.0357141 ... -2.0357141 -2.0357141 -2.0357141]
  [-2.0357141 -2.0357141 -2.0357141 ... -2.0357141 -2.0357141 -2.0357141]
  [-2.0357141 -2.0357141 -2.0357141 ... -2.0357141 -2.0357141 -2.0357141]
  ...
  [-2.0357141 -2.0357141 -2.0357141 ... -2.0357141 -2.0357141 -2.0357141]
  [-2.0357141 -2.0357141 -2.0357141 ... -2.0357141 -2.0357141 -2.0357141]
  [-2.0357141 -2.0357141 -2.0357141 ... -2.0357141 -2.03571

In [12]:
vgg16 = tv.models.vgg16_bn(weights=tv.models.VGG16_BN_Weights.DEFAULT)

for param in vgg16.features.parameters():
    param.requires_grad = False

num_features = vgg16.classifier[-1].in_features
features = list(vgg16.classifier.children())[:-1]
features.extend([nn.Linear(num_features, 2)])
vgg16.classifier = nn.Sequential(*features)

print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [13]:
vgg16.cuda()

criterion = nn.CrossEntropyLoss().to(device)
# optimizer = torch.optim.RMSprop(lr=1e-4)
optimizer = torch.optim.SGD(vgg16.parameters(), lr=1e-3, momentum=0.9)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [14]:
def test_model(model, criterion, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            del data, target, output, pred
            torch.cuda.empty_cache()
    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )


def train_epoch(model, criterion, device, train_loader, optimizer, epoch):
    model.train()
    model.to(device)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability
        correct = pred.eq(target.view_as(pred)).sum().item()

        data_len = len(data)
        del data, target, output, pred
        torch.cuda.empty_cache()

        if batch_idx % 15 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy: {:.2f}%".format(
                    epoch,
                    batch_idx * data_len,
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                    100.0 * correct / data_len,
                )
            )


def train_model(
    model,
    train_dataloader,
    val_dataloader,
    optimizer,
    criterion,
    scheduler,
    num_epochs=25,
):
    model = model.to(device)
    for epoch in range(1, num_epochs):
        train_epoch(model, criterion, device, train_dataloader, optimizer, epoch)
        test_model(model, criterion, device, val_dataloader)
        scheduler.step()
        print("=" * 30)


train_model(
    vgg16,
    train_dataloader,
    val_dataloader,
    optimizer,
    criterion,
    exp_lr_scheduler,
    num_epochs=200,
)

Train Epoch: 1 [0/192 (0%)]	Loss: 0.618244	 Accuracy: 65.62%

Test set: Average loss: 0.0464, Accuracy: 31/51 (61%)

Train Epoch: 2 [0/192 (0%)]	Loss: 0.632570	 Accuracy: 62.50%

Test set: Average loss: 0.0457, Accuracy: 37/51 (73%)

Train Epoch: 3 [0/192 (0%)]	Loss: 0.695545	 Accuracy: 46.88%

Test set: Average loss: 0.0400, Accuracy: 40/51 (78%)

Train Epoch: 4 [0/192 (0%)]	Loss: 0.491938	 Accuracy: 75.00%

Test set: Average loss: 0.0410, Accuracy: 40/51 (78%)

Train Epoch: 5 [0/192 (0%)]	Loss: 0.537890	 Accuracy: 71.88%

Test set: Average loss: 0.0409, Accuracy: 40/51 (78%)

Train Epoch: 6 [0/192 (0%)]	Loss: 0.477790	 Accuracy: 71.88%

Test set: Average loss: 0.0392, Accuracy: 41/51 (80%)

Train Epoch: 7 [0/192 (0%)]	Loss: 0.463929	 Accuracy: 75.00%

Test set: Average loss: 0.0337, Accuracy: 41/51 (80%)

Train Epoch: 8 [0/192 (0%)]	Loss: 0.417252	 Accuracy: 78.12%

Test set: Average loss: 0.0286, Accuracy: 42/51 (82%)

Train Epoch: 9 [0/192 (0%)]	Loss: 0.175891	 Accuracy: 100.00%

T